In [15]:
import numpy as np


class LogisticRegression():
    @staticmethod
    def loss(theta, x, y, lambda_param=None):
        """Loss function for logistic regression with without regularization"""
        exponent = - y * (x.dot(theta))
        return np.sum(np.log(1+np.exp(exponent))) / x.shape[0]

    @staticmethod
    def gradient(theta, x, y, lambda_param=None):
        """
        Gradient function for logistic regression without regularization.
        Based on the above logistic_regression
        """
        exponent = y * (x.dot(theta))
        gradient_loss = - (np.transpose(x) @ (y / (1+np.exp(exponent)))) / (
            x.shape[0])

        # Reshape to handle case where x is csr_matrix
        gradient_loss.reshape(theta.shape)

        return gradient_loss


class LogisticRegressionSinglePoint():
    @staticmethod
    def loss(theta, xi, yi, lambda_param=None):
        exponent = - yi * (xi.dot(theta))
        return np.log(1 + np.exp(exponent))

    @staticmethod
    def gradient(theta, xi, yi, lambda_param=None):

        # Based on page 22 of
        # http://www.cs.rpi.edu/~magdon/courses/LFD-Slides/SlidesLect09.pdf
        exponent = yi * (xi.dot(theta))
        return - (yi*xi) / (1+np.exp(exponent))


class LogisticRegressionRegular():
    @staticmethod
    def loss(theta, x, y, lambda_param):
        regularization = (lambda_param/2) * np.sum(theta*theta)
        return LogisticRegression.loss(theta, x, y) + regularization

    @staticmethod
    def gradient(theta, x, y, lambda_param):
        regularization = lambda_param * theta
        return LogisticRegression.gradient(theta, x, y) + regularization

In [5]:
'''
Functions from in-class exercises
'''
# Load the data and libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def laplace_mech_vec(vec, sensitivity, epsilon):
    return [v + np.random.laplace(loc=0, scale=sensitivity / epsilon) for v in vec]

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

In [6]:
# Load data files
import numpy as np
import urllib.request
import io

url_x = 'https://github.com/jnear/cs211-data-privacy/raw/master/slides/adult_processed_x.npy'
url_y = 'https://github.com/jnear/cs211-data-privacy/raw/master/slides/adult_processed_y.npy'

with urllib.request.urlopen(url_x) as url:
    f = io.BytesIO(url.read())
X = np.load(f)

with urllib.request.urlopen(url_y) as url:
    f = io.BytesIO(url.read())
y = np.load(f)

In [7]:
# Split data into training and test sets
training_size = int(X.shape[0] * 0.8)

X_train = X[:training_size]
X_test = X[training_size:]

y_train = y[:training_size]
y_test = y[training_size:]

print('Train and test set sizes:', len(y_train), len(y_test))

Train and test set sizes: 36176 9044


In [63]:
'''
Functions taken from in-class-exercise 10.28.24
'''
# Prediction: take a model (theta) and a single example (xi) and return its predicted label
def predict(xi, theta, bias=0):
    label = np.sign(xi @ theta + bias) #this is the dot product and take the sign. 
    return label

def accuracy(theta):
    return np.sum(predict(X_test, theta) == y_test)/X_test.shape[0]

def L2_clip(v, b):
    norm = np.linalg.norm(v, ord=2) #computing L2 norm 
    
    if norm > b:
        return b * (v / norm)
    else:
        return v

def gradient_sum(theta, X, y, b):
    gradients = [L2_clip(gradient(theta, x_i, y_i), b) for x_i, y_i in zip(X,y)]
        
    # sum query
    # L2 sensitivity is b (by clipping performed above)
    return np.sum(gradients, axis=0)
#theta = [-.1 for _ in range(104)]
#accuracy(theta)

### IMPLEMENTING BATCH GRADIENT DESCENT ###

In [123]:
def acuracy(theta):
    predictions = np.sign(X_train.dot(theta))  # Apply the sign function to get class predictions
    return np.mean(predictions == y_train)

def batch_gradient_descent(epochs, n):
    
    theta = np.zeros(X_train.shape[1])
    regression = LogisticRegression()
    
    # Loop through epochs
    for _ in range(epochs):
        # Compute gradient using the vectorized function
        
        grad = regression.gradient(theta, X_train, y_train, lambda_param=None) 

        # Update the parameters (theta)
        theta -= n * grad

    #print(theta)
    return theta

theta0 = batch_gradient_descent(20, .1)
theta1 = batch_gradient_descent(40, .1)
theta2 = batch_gradient_descent(60, .1)
theta3 = batch_gradient_descent(80, .1)
theta4 = batch_gradient_descent(100, .1)
theta5 = batch_gradient_descent(20, 1)
theta6 = batch_gradient_descent(40, 1)
theta7 = batch_gradient_descent(60, 1)
theta8 = batch_gradient_descent(80, 1)
theta9 = batch_gradient_descent(100, 1)

print('Final accuracy with epochs = 20, learning rate = .1:  ', accuracy(theta0))
print('Final accuracy with epochs = 40, learning rate = .1:  ', accuracy(theta1))
print('Final accuracy with epochs = 60, learning rate = .1:  ', accuracy(theta2))
print('Final accuracy with epochs = 80, learning rate = .1:  ', accuracy(theta3))
print('Final accuracy with epochs = 100, learning rate = .1:  ', accuracy(theta4))
print('Final accuracy with epochs = 20, learning rate = 1:  ', accuracy(theta5))
print('Final accuracy with epochs = 40, learning rate = 1:  ', accuracy(theta6))
print('Final accuracy with epochs = 60, learning rate = 1:  ', accuracy(theta7))
print('Final accuracy with epochs = 80, learning rate = 1:  ', accuracy(theta8))
print('Final accuracy with epochs = 100, learning rate = 1:  ', accuracy(theta9))

Final accuracy with epochs = 20, learning rate = .1:   0.7585139318885449
Final accuracy with epochs = 40, learning rate = .1:   0.7587350729765591
Final accuracy with epochs = 60, learning rate = .1:   0.7628261831048209
Final accuracy with epochs = 80, learning rate = .1:   0.7778637770897833
Final accuracy with epochs = 100, learning rate = .1:   0.7807386112339673
Final accuracy with epochs = 20, learning rate = 1:   0.8081601061477223
Final accuracy with epochs = 40, learning rate = 1:   0.8159000442282176
Final accuracy with epochs = 60, learning rate = 1:   0.8182220256523662
Final accuracy with epochs = 80, learning rate = 1:   0.8209862892525431
Final accuracy with epochs = 100, learning rate = 1:   0.8228659885006634


### EPSILON DP ###

In [35]:
def batch_gradient_epsilon_descent(epochs, epsilon, n):
    
    theta = np.zeros(X_train.shape[1])
    regression = LogisticRegression()
    epsilon_i = epsilon/epochs
    
    # Loop through epochs
    for _ in range(epochs):
        # Compute gradient using the vectorized function
        
        grad = regression.gradient(theta, X_train, y_train, lambda_param=None)
        grad_noisy = laplace_mech_vec(grad, sensitivity=1, epsilon=epsilon_i)
        noisy_array = np.array(grad_noisy)

        # Update the parameters (theta)
        theta -= n * noisy_array

    print(theta)
    return theta

theta = batch_gradient_epsilon_descent(50, 100, 1) #Needs a high epsilon
theta
accuracy(theta)

[  5.23815975   5.78745617   1.00852662  -1.75662619  -3.41198047
  -4.80469834   6.64370471  -0.86000989   2.62906297   0.76557664
   0.53598117   1.42502169   3.97435256   1.48369348   6.39285508
  -0.62831194   6.79219427   5.02449443   0.96698887  -4.51679907
  -0.58858787  -0.96942414  10.62507056  -0.46960108  -1.6105687
  -3.09863144   4.13230502  -4.7383503    1.59448386  -2.1213927
  -6.77310865   5.33738851   3.97783674  -8.80844523  -0.401746
   1.71818944   3.43665327 -14.22137987   1.57265143  -0.308636
  -1.97016339  -1.59255716  -0.22005702   8.39779865  -0.62351416
 -10.18477711  -9.02416568   2.04287195  -7.53609551   1.43248538
   3.58755683   2.02823716   0.14902395   2.07261662  -6.21659812
   6.02064628  10.06618258  -1.26192237   3.39608873   4.89034576
   1.49755143  -0.19261116   2.78852041  -6.58004916 -14.31835326
  -8.19017156   3.36573215   6.73976769  -6.16409321   1.28512722
  -1.65828633   4.61174212   3.93692728  -7.62879656  -4.66214534
  -2.26875856  1

0.5699911543564794

### EPSILON DELTA DP ###

In [198]:
def batch_gradient_epsilon_delta_descent(epochs, epsilon, delta, n):
    
    theta = np.zeros(X_train.shape[1])
    regression = LogisticRegression()
    epsilon_i = epsilon/epochs
    delta_i = delta/epochs
    b=3
    
    # Loop through epochs
    for _ in range(epochs):
        # Compute gradient using the vectorized function
        
        grad = regression.gradient(theta, X_train, y_train, lambda_param=None)
        clipped_grad = L2_clip(grad, b)
        grad_noisy = gaussian_mech_vec(clipped_grad, sensitivity=b, epsilon=epsilon_i, delta = delta_i)
        noisy_array = np.array(grad_noisy)

        # Update the parameters (theta)
        theta -= n * noisy_array

    #print(theta)
    return theta

theta = batch_gradient_epsilon_delta_descent(50, 100, 1e-5, 1) #Needs a high epsilon
theta
accuracy(theta)

[ -49.41731588   30.37049136   11.45693737  -30.20274393  -88.07933288
  -35.35845856   14.28665606  100.68227408  -23.140508    -47.3081694
   -7.35290573   52.39077016    8.94129519   53.58991605   24.79771902
  -97.65490888  102.07186543  -73.71000536  -32.28139141  -22.90670257
   -6.51303884  -84.77268749   18.2648349    55.3874417   -30.96521827
  105.19868052  -93.74657329    7.69245444  -39.24928539   21.24814705
  -49.33031786  -23.62941913   21.40154993  -14.36268552   81.80375251
  -24.17940414  -69.11407754  -30.24454097   69.75002965   -6.95658538
  -35.5212063    59.76883197   58.23824783  117.32047512  -67.45789463
  -16.68675865   42.89123458  -21.2459076    36.41038241  -94.41760507
 -129.15517725   81.5388054   -30.41120409  -38.96166682   29.41256427
   50.82288111 -124.8568815    23.92797033   76.02150176    6.37276324
 -105.18611626  -92.50859826 -104.23732649   43.67335521 -112.38387683
   66.36136194  -70.56529625  -41.47587296  -60.67434777 -100.68379199
   68.8

0.5222246793454224

### RDP ###

In [224]:
def gaussian_mech_RDP_vec(vec, sensitivity, alpha, epsilon):
    sigma = np.sqrt((sensitivity**2 * alpha) / (2 * epsilon))
    return [v + np.random.normal(loc=0, scale=sigma) for v in vec]

def batch_gradient_rdp_descent(epochs, epsilon_bar, alpha, n):
    
    theta = np.zeros(X_train.shape[1])
    
    regression = LogisticRegression()
    
    epsilon_i = epsilon_bar/epochs
    alpha_i = alpha/epochs
    b=3
    
    # Loop through epochs
    for _ in range(epochs):
        # Compute gradient using the vectorized function
        
        grad = regression.gradient(theta, X_train, y_train, lambda_param=None)
        clipped_grad = L2_clip(grad, b)
        grad_noisy = gaussian_mech_RDP_vec(clipped_grad, sensitivity=b, alpha=alpha_i, epsilon=epsilon_i)
        noisy_array = np.array(grad_noisy)

        # Update the parameters (theta)
        theta -= n * noisy_array

    print(theta)
    return theta

theta = batch_gradient_rdp_descent(50, 100, 10, 1) #Needs a high epsilon
theta
accuracy(theta)

[-2.54657349  5.02974815  2.35003439 -0.01703512  0.28942141 -2.61561595
 -5.26921933 -2.43695499  8.46912989  1.76271193 -4.65601623 -3.00029351
  2.29102998  3.45498216 -0.75804558  1.25043994  1.12620038  1.4250339
 -5.12023657  3.17958699 -1.35816615  2.5008328  -0.80356306  3.96753397
 -7.64242691  0.63116457 -0.05379498 -2.46367853  6.81642045  0.68310843
  4.0034507   5.86168102 -1.38098818  0.0205345   1.16443706 -6.20750007
 -1.96976079  0.9722609  -1.87762097 -0.60806927 -1.96562999  4.45232717
  0.56502517 -0.64752587 -0.60460136 -1.37211564 -0.16804562 -3.11666514
 -4.0186661   1.91260285  2.75676419  9.94573596 -1.87167689  1.65947714
 -1.42124375 -2.34339042  3.70056675 -0.47323726  0.77364924 -2.4619316
  2.82227807 -0.86610472 -7.85214669  3.70250752  3.34593545 -0.64976129
  1.73212049  3.32917508  1.61334875 -2.4505212  -0.35716404  5.77131929
  7.55902796  0.39753227  9.68056554 -2.23330871  3.04656239  1.12539396
  0.80675537  2.42483223 -5.16090887 -6.46953632 -6.1

0.706766917293233

### zCDP ###

In [261]:
def gaussian_mech_zCDP_vec(vec, sensitivity, rho):
    sigma = np.sqrt((sensitivity**2) / (2 * rho))
    return [v + np.random.normal(loc=0, scale=sigma) for v in vec]

def batch_gradient_zcdp_descent(epochs, rho, n):
    
    theta = np.zeros(X_train.shape[1])
    
    regression = LogisticRegression()
    
    rho_i = rho/epochs
    b=3
    
    # Loop through epochs
    for _ in range(epochs):
        # Compute gradient using the vectorized function
        
        grad = regression.gradient(theta, X_train, y_train, lambda_param=None)
        clipped_grad = L2_clip(grad, b)
        grad_noisy = gaussian_mech_zCDP_vec(clipped_grad, sensitivity=b, rho=rho_i)
        noisy_array = np.array(grad_noisy)

        # Update the parameters (theta)
        theta -= n * noisy_array

    print(theta)
    return theta

theta = batch_gradient_zcdp_descent(50, 100, 1) #Needs a high epsilon
theta
accuracy(theta)

[ -1.46107744  18.71017285   7.28563567  -4.54468891   5.01906203
   7.59812795   6.36195557   7.33400508  -6.95330201  -2.26451643
   3.37230708   9.16754422  -9.52830718  -4.09790484   0.93176367
  -8.97875391 -18.70332643   2.98311248   2.53154384  -4.33785959
  10.09436114 -10.83627957   6.46440944  -0.39808524  -6.30948589
  17.01794052   8.43970677  -6.60103456  -4.07385403   0.17903216
  -5.15959474  -0.41420193  -2.35377986   9.09503676  -2.79623215
   1.08663891   1.19205575  -7.285804   -14.6534904   -5.38105877
  16.79749162   4.60229295   9.39628074  -1.82150649  -3.92576991
 -13.73777097 -13.31216002  -8.07285285  -3.93243221  15.51058605
   7.16723023   6.84648798   6.21616575   1.46579858  -2.43688902
  -7.37710042 -11.01506289   4.53785564   7.90284772 -12.62762025
   6.74948835  19.5802878   -5.92765874   6.48700362 -13.39905439
  16.34471642  -5.01908456  -5.02743027  10.07750738 -29.40813581
   0.36457593   9.34901734   9.36591867   5.76699937  12.71497993
  -4.80664

0.7289915966386554

In [263]:
# Moving forward - need to convert to epsilon-delta for zCDP and RDP to determine 
# which produces the best accuracy